# **DATA UTILS**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import math
import os

# plot history for accuracy
def plot_acc(history, title = None):
    fig = plt.figure()
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    if title is not None:
        plt.title(title)

    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc=0)

# plot history for loss
def plot_loss(history, title = None):
    fig = plt.figure()
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    if title is not None:
        plt.title(title)

    plt.ylabel('Loss')
    plt.xlabel('Epoch')

    plt.legend(['Train', 'Val'], loc=0)

# show results using each model
def show_result(model, x, y, title = 'Result', save_file = False, save_file_name = 'show_result', IMG_HEIGHT = 384, IMG_WIDTH = 256):
    if len(x) != len(y):
        raise Exception('Check the number of images and labels')
    else:
        pass

    x = x.reshape([1, IMG_HEIGHT, IMG_WIDTH, 3])
    pred = model.predict(x)
    pred = np.argmax(pred, axis = 3)

    y = y.reshape([1, IMG_HEIGHT, IMG_WIDTH, 4])
    gt = np.argmax(y, axis = 3)

    fig = plt.figure(figsize = (12, 5))
    plt.suptitle(title, fontsize=25)
    plt.subplot(1,4,1)
    plt.imshow(np.squeeze(x))
    plt.title('Image')
    plt.subplot(1,4,2)
    plt.imshow(np.squeeze(gt))
    plt.title('Ground truth')
    plt.subplot(1,4,3)
    plt.imshow(np.squeeze(pred))
    plt.title('Prediction')
    plt.subplot(1,4,4)
    plt.imshow(np.squeeze(x))
    masked_imclass = np.ma.masked_where(np.squeeze(pred) == 0, np.squeeze(pred))
    plt.imshow( masked_imclass, alpha = 0.4)
    plt.title('Overlay')
    plt.show()

    # save results
    if save_file == True:
        save_dir = './result/show_result/'
        if not os.path.exists(save_dir): # if there is no exist, make the path
            os.makedirs(save_dir)
        fig.savefig(save_dir + save_file_name)
    return

# save image segmented by skin, hair, clothes
def iamge_segmentation(model, x, title = 'Each Segmentation',save_file = False, save_file_name = 'iamge_segmentation', IMG_HEIGHT = 384, IMG_WIDTH = 256):
    x = x.reshape([1, IMG_HEIGHT, IMG_WIDTH, 3])

    pred = model.predict(x)
    pred = np.argmax(pred, axis = 3)
    empty = np.zeros((pred.shape))

    fig = plt.figure(figsize = (12, 5))
    plt.subplot(1,4,1)
    plt.imshow(np.squeeze(x))
    plt.title('Image')
    plt.subplot(1,4,2)
    plt.imshow(np.squeeze(x))
    masked_imclass = np.ma.masked_where(np.squeeze(pred) == 1, np.squeeze(empty))
    plt.imshow( masked_imclass, alpha = 1)
    plt.title('Skin')
    plt.subplot(1,4,3)
    plt.imshow(np.squeeze(x))
    masked_imclass = np.ma.masked_where(np.squeeze(pred) == 2, np.squeeze(empty))
    plt.imshow( masked_imclass, alpha = 1)
    plt.title('Hair')
    plt.subplot(1,4,4)
    plt.imshow(np.squeeze(x))
    masked_imclass = np.ma.masked_where(np.squeeze(pred) == 3, np.squeeze(empty))
    plt.imshow( masked_imclass, alpha = 1)
    plt.title('Clothes')
    plt.show()
    if save_file == True:
        save_dir = './result/iamge_segmentation/'
        if not os.path.exists(save_dir): # if there is no exist, make the path
            os.makedirs(save_dir)
        fig.savefig(save_dir + save_file_name)

    return

# **MODEL AND FUNCTIONS**



In [ ]:
import keras
from keras import models
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, Input, Concatenate, Conv2DTranspose
from keras.optimizers import Adam
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import os

class UNet(object):
    def __init__(self, img_shape, num_of_class, actf = 'relu',
        learning_rate = 0.001,  drop_rate = 0.5, do_batch_norm = False, do_drop = False):
        self.learning_rate = learning_rate
        self.actf = actf
        self.img_shape = img_shape
        self.num_of_class = num_of_class
        self.drop_rate = drop_rate
        self.do_batch_norm = do_batch_norm
        self.do_drop = do_drop

        self.model = self.build_model()

    # encoding block(conv - conv - pool)
    def enc_conv_block(self, inputs, feature_maps, filter_size = (3, 3),
                           conv_strides = 1, pooling_filter_size = (2, 2), pooling_strides = (2, 2)):
        conv1 = Conv2D(feature_maps , filter_size , activation = self.actf, strides = conv_strides,
                           padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv2 = Conv2D(feature_maps , filter_size , activation = self.actf, strides = conv_strides,
                           padding = 'same', kernel_initializer = 'he_normal')(conv1)
        pool = MaxPooling2D(pooling_filter_size, strides = pooling_strides)(conv2)

        return pool, conv2

    # decoding block(concat - upconv - upconv)
    def dec_conv_block(self, inputs, merge_inputs, feature_maps, filter_size = (3, 3), conv_strides = 1,
                           up_conv_strides = (2, 2)):

        merge = Concatenate(axis = 3)([Conv2DTranspose(feature_maps, filter_size,
                                                       activation = self.actf, strides = up_conv_strides, kernel_initializer = 'he_normal',
                                                       padding = 'same')(inputs), merge_inputs])

        conv1 = Conv2D(feature_maps , filter_size , activation = self.actf, strides = conv_strides,
                           padding = 'same', kernel_initializer = 'he_normal')(merge)
        conv2 = Conv2D(feature_maps , filter_size , activation = self.actf, strides = conv_strides,
                           padding = 'same', kernel_initializer = 'he_normal')(conv1)

        return conv2

    # encoder
    def encoding_path(self, inputs):

        enc_conv1, concat1 = self.enc_conv_block(inputs, 64)
        enc_conv2, concat2 = self.enc_conv_block(enc_conv1, 128)
        enc_conv3, concat3 = self.enc_conv_block(enc_conv2, 256)
        enc_conv4, concat4 = self.enc_conv_block(enc_conv3, 512)

        return concat1, concat2, concat3, concat4, enc_conv4

    # decoder
    def decoding_path(self, dec_inputs, concat1, concat2, concat3, concat4):

        dec_conv1 = self.dec_conv_block(dec_inputs, concat4, 512)
        dec_conv2 = self.dec_conv_block(dec_conv1, concat3, 256)
        dec_conv3 = self.dec_conv_block(dec_conv2, concat2, 128)
        dec_conv4 = self.dec_conv_block(dec_conv3, concat1, 64)

        return dec_conv4
    # build network
    def build_model(self):
        inputs = Input(self.img_shape)

        # Contracting path
        concat1, concat2, concat3, concat4, enc_path = self.encoding_path(inputs)

        # middle path
        mid_path1 = Conv2D(1024, (3,3), activation = self.actf, padding = 'same', kernel_initializer = 'he_normal')(enc_path)
        mid_path1 = Dropout(self.drop_rate)(mid_path1)
        mid_path2 = Conv2D(1024, (3,3), activation = self.actf, padding = 'same', kernel_initializer = 'he_normal')(mid_path1)
        mid_path2 = Dropout(self.drop_rate)(mid_path2)

        # Expanding path
        dec_path = self.decoding_path(mid_path2, concat1, concat2, concat3, concat4)
        segmented = Conv2D(self.num_of_class, (1,1), activation = self.actf, padding = 'same', kernel_initializer = 'he_normal')(dec_path)
        segmented = Activation('softmax')(segmented)

        model = Model(inputs = inputs, outputs = segmented)
        model.compile(optimizer = Adam(lr = self.learning_rate),
                          loss = 'categorical_crossentropy', metrics = ['accuracy'])

        return model

    # train model
    def train(self, X_train, Y_train, epoch = 10, batch_size = 32, val_split = 0.2, shuffle = True):

        self.history = self.model.fit(X_train, Y_train, validation_split = val_split,
                                          epochs = epoch, batch_size = batch_size, shuffle =  shuffle)
        return self.history

    # train with data augmentation
    def train_generator(self, x_train, y_train, x_test, y_test, name_model, epoch = 10, batch_size = 32, val_split = 0.2, min_lr = 1e-06):

        train_datagen = ImageDataGenerator(
            rescale=1./255,
            brightness_range=[0.7, 1.3]
        )

        val_datagen = ImageDataGenerator(
            rescale=1./255
        )

        train_gen = train_datagen.flow(
            x_train,
            y_train,
            batch_size = batch_size,
            shuffle=True
        )

        val_gen = val_datagen.flow(
            x_test,
            y_test,
            batch_size = batch_size,
            shuffle=False
        )

        save_dir = './save_model/'
        if not os.path.exists(save_dir): # if there is no exist, make the path
            os.makedirs(save_dir)

        cb_checkpoint = ModelCheckpoint(save_dir + name_model + '.h5', monitor = 'val_acc', save_best_only = True, verbose = 1)
        reduce_lr = ReduceLROnPlateau(monitor = 'val_acc', factor = 0.2, patience = 5, verbose = 1, min_lr = min_lr)

        self.history = self.model.fit_generator(train_gen,
                                                validation_data=val_gen,
                                                epochs=epoch,
                                                callbacks=[cb_checkpoint, reduce_lr])
        return self.history
    # predict test data
    def predict(self, X_test):
        pred_classes = self.model.predict(X_test)

        return pred_classes

    # show architecture
    def show_model(self):
        return print(self.model.summary())

    # reuse model
    def saved_model_use(self, save_dir = None):
        if save_dir == None:
            return print('No path')

        self.model.load_weights(save_dir)

        return print("Loaded model from '{}'".format(save_dir))


In [ ]:
import numpy as np
IMG_HEIGHT = 384
IMG_WIDTH = 256
BATCH_SIZE = 8

x_train = np.load('../input/clothesdata/datasets/x_train.npy')
x_test = np.load('../input/clothesdata/datasets/x_test.npy')
y_train = np.load('../input/clothesdata/datasets/y_train_onehot.npy')
y_test = np.load('../input/clothesdata/datasets/y_test_onehot.npy')

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
model = UNet(img_shape = x_train[0].shape, num_of_class = 4,learning_rate = 1e-3)

In [ ]:
model.show_model()

In [ ]:
history = model.train_generator(x_train, y_train, 
                                x_test, y_test, 
                                'UNet_model',
                                epoch = 50,
                                batch_size = BATCH_SIZE)